### Loading Datasets

In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.ExcelFile('../dataset/global_superstore_2016.xlsx')

In [6]:
df.sheet_names

['Orders', 'Returns', 'People']

In [7]:
dfs = {sheet: df.parse(sheet) for sheet in df.sheet_names}
orders, returns, people = dfs['Orders'], dfs['Returns'], dfs['People']

### 1. Cleaning on Orders

In [8]:
orders.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium


In [9]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
Row ID            51290 non-null int64
Order ID          51290 non-null object
Order Date        51290 non-null datetime64[ns]
Ship Date         51290 non-null datetime64[ns]
Ship Mode         51290 non-null object
Customer ID       51290 non-null object
Customer Name     51290 non-null object
Segment           51290 non-null object
Postal Code       9994 non-null float64
City              51290 non-null object
State             51290 non-null object
Country           51290 non-null object
Region            51290 non-null object
Market            51290 non-null object
Product ID        51290 non-null object
Category          51290 non-null object
Sub-Category      51290 non-null object
Product Name      51290 non-null object
Sales             51290 non-null float64
Quantity          51290 non-null int64
Discount          51290 non-null float64
Profit            51290 non-null fl

In [10]:
orders.columns = ['Row_ID', 'Order_ID', 'Order_Date', 'Ship_Date', 'Ship_Mode','Customer_ID', 'Customer_Name', 
                  'Segment', 'Postal_Code', 'City', 'State', 'Country', 'Region', 'Market', 'Product_ID', 'Category',
                  'Sub-Category', 'Product_Name', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping_Cost', 'Order_Priority']

In [11]:
# Drop postal code since the dataset contains different countries
orders = orders.drop(['Postal_Code', 'Row_ID'], axis=1)
# Add processing time
orders['Processing_Time_days'] = orders['Ship_Date'] - orders['Order_Date']
orders['Processing_Time_days'] = [int(day.days) for day in orders.Processing_Time_days]

In [12]:
orders.to_csv('orders.csv')

### 2. Cleaning on Returns

In [13]:
returns.head(3)

,Returned,Order ID,Region
0,Yes,CA-2012-SA20830140-41210,Central US
1,Yes,IN-2012-PB19210127-41259,Eastern Asia
2,Yes,CA-2012-SC20095140-41174,Central US


In [14]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 3 columns):
Returned    1079 non-null object
Order ID    1079 non-null object
Region      1079 non-null object
dtypes: object(3)
memory usage: 25.4+ KB


In [15]:
returns.columns = ['Returned', 'Order_ID', 'Region']
returns.drop('Region', axis=1, inplace=True)

In [16]:
returns.head()

,Returned,Order_ID
0,Yes,CA-2012-SA20830140-41210
1,Yes,IN-2012-PB19210127-41259
2,Yes,CA-2012-SC20095140-41174
3,Yes,IN-2015-JH158207-42140
4,Yes,IN-2014-LC168857-41747


In [69]:
returns.to_csv('returns.csv')

In [17]:
merge_with_returns = orders.merge(returns, left_on='Order_ID', right_on='Order_ID', how='left')
merge_with_returns['Returned'] = [np.where(returned=='Yes', 1, 0) for returned in merge_with_returns.Returned]

### 3. Get income level for each customer by country

* Low income: USD 995 or LESS <br>
* Lower middle income: USD 996 to USD 3,895 <br>
* Upper middle income: USD 3,896 to USD 12,055 <br>
* High income: USD 12,056 or MORE

In [20]:
df2 = pd.ExcelFile('../dataset/CLASS.xls')
df2.sheet_names

['Parameters',
 'Thresholds',
 'Country Analytical History',
 'Operational Category Changes',
 'Country Indebtedness History']

In [21]:
dfs2 = {sheet: df2.parse(sheet) for sheet in df2.sheet_names}
income_level_tt = dfs2['Country Analytical History']


In [22]:
income_level_tt.rename(columns=income_level_tt.iloc[4],inplace=True)

In [23]:
income_level_tt.drop(income_level_tt.index[0:15],inplace=True)

In [34]:
income_level_tt.rename(columns = {2015:'Income_level'},inplace = True)

In [35]:
income_level_copy = income_level_tt[['Data for calendar year :','Income_level']].copy()

In [36]:
income_level_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224 entries, 15 to 238
Data columns (total 2 columns):
Data for calendar year :    221 non-null object
Income_level                213 non-null object
dtypes: object(2)
memory usage: 5.2+ KB


In [37]:
income_level_copy.rename(columns = {'Data for calendar year :':'Country'},inplace = True)

In [38]:
income_level_copy =income_level_copy.reset_index()

In [39]:
income_level = income_level_copy[['Country','Income_level']].copy()

In [40]:
income_level.to_csv('income_level.csv')

In [41]:
merge_with_returns_income = merge_with_returns.merge(income_level, left_on='Country', right_on='Country', how='left')

In [42]:
merge_with_returns_income.head()

,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,City,State,Country,...,Product_Name,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Processing_Time_days,Returned,Income_level
0,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,Oklahoma City,Oklahoma,United States,...,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High,2,0,H
1,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,...,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,2,0,H
2,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,Brisbane,Queensland,Australia,...,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,1,0,H
3,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,Berlin,Berlin,Germany,...,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,2,0,H
4,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,Dakar,Dakar,Senegal,...,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical,1,0,L


In [43]:
merge_with_returns_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51290 entries, 0 to 51289
Data columns (total 25 columns):
Order_ID                51290 non-null object
Order_Date              51290 non-null datetime64[ns]
Ship_Date               51290 non-null datetime64[ns]
Ship_Mode               51290 non-null object
Customer_ID             51290 non-null object
Customer_Name           51290 non-null object
Segment                 51290 non-null object
City                    51290 non-null object
State                   51290 non-null object
Country                 51290 non-null object
Region                  51290 non-null object
Market                  51290 non-null object
Product_ID              51290 non-null object
Category                51290 non-null object
Sub-Category            51290 non-null object
Product_Name            51290 non-null object
Sales                   51290 non-null float64
Quantity                51290 non-null int64
Discount                51290 non-null float64


### 4. Get gender for each customer

https://github.com/miriamposner/derive_gender/blob/master/derive-gender-from-a-column-of-first-names.md

In [44]:
first_name = pd.Series([name.split()[0] for name in orders['Customer_Name']]).unique()
pd.DataFrame(first_name).to_csv('first_name.csv')

In [45]:
gender_tt = pd.read_csv('genderize.csv')

In [46]:
gender_tt.rename(columns = {0:'First_Name'},inplace = True)

In [47]:
gender = gender_tt[['0','gender']].copy()

In [48]:
gender.columns = ['First_Name', 'Gender']

In [49]:
gender['Gender'] = [gender.replace("\"", "") for gender in gender.Gender]
gender['Gender'] = [np.where(gender=='female', 1, 0) for gender in gender.Gender]

In [50]:
merge_with_returns_income['First_Name'] = [name.split()[0] for name in merge_with_returns_income['Customer_Name']]

In [51]:
merge_with_returns_income_gender =  merge_with_returns_income.merge(gender, left_on='First_Name', right_on='First_Name', how='left')
merge_with_returns_income_gender.drop('First_Name', axis=1, inplace=True)

In [52]:
merge_with_returns_income_gender.head()

,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,City,State,Country,...,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Processing_Time_days,Returned,Income_level,Gender
0,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,Oklahoma City,Oklahoma,United States,...,221.980,2,0.0,62.1544,40.77,High,2,0,H,NaN
1,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,...,3709.395,9,0.1,-288.7650,923.63,Critical,2,0,H,NaN
2,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,Brisbane,Queensland,Australia,...,5175.171,9,0.1,919.9710,915.49,Medium,1,0,H,NaN
3,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,Berlin,Berlin,Germany,...,2892.510,5,0.1,-96.5400,910.16,Medium,2,0,H,NaN
4,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,Dakar,Dakar,Senegal,...,2832.960,8,0.0,311.5200,903.04,Critical,1,0,L,NaN


In [53]:
merge_with_returns_income_gender.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51290 entries, 0 to 51289
Data columns (total 26 columns):
Order_ID                51290 non-null object
Order_Date              51290 non-null datetime64[ns]
Ship_Date               51290 non-null datetime64[ns]
Ship_Mode               51290 non-null object
Customer_ID             51290 non-null object
Customer_Name           51290 non-null object
Segment                 51290 non-null object
City                    51290 non-null object
State                   51290 non-null object
Country                 51290 non-null object
Region                  51290 non-null object
Market                  51290 non-null object
Product_ID              51290 non-null object
Category                51290 non-null object
Sub-Category            51290 non-null object
Product_Name            51290 non-null object
Sales                   51290 non-null float64
Quantity                51290 non-null int64
Discount                51290 non-null float64


In [54]:
merge_with_returns_income_gender.to_csv('new.csv')

### 5. Group by category and order date

In [55]:
merge_with_returns_income_gender.Category.unique()

array(['Technology', 'Furniture', 'Office Supplies'], dtype=object)

In [56]:
technology = merge_with_returns_income_gender[merge_with_returns_income_gender.Category == 'Technology']
furniture = merge_with_returns_income_gender[merge_with_returns_income_gender.Category == 'Furniture']
office_supplies = merge_with_returns_income_gender[merge_with_returns_income_gender.Category == 'Office Supplies']

In [57]:
ts_all = merge_with_returns_income_gender.groupby('Order_Date')['Sales'].sum()
ts_all.to_csv('ts_all.csv')

In [58]:
ts_technology = technology.groupby('Order_Date')['Sales'].sum()
ts_furniture = furniture.groupby('Order_Date')['Sales'].sum()
ts_office_supplies = office_supplies.groupby('Order_Date')['Sales'].sum()
ts_technology.to_csv('ts_technology.csv')
ts_furniture.to_csv('ts_furniture.csv')
ts_office_supplies.to_csv('ts_office_supplies.csv')